In [17]:
import numpy as np
import pandas as pd
import requests
import json
import os
import dotenv
import sys
sys.tracebacklimit = 0 # turn off the error tracebacks
from newspaper import Article
import re
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

In [18]:
dotenv.load_dotenv()
token = os.getenv('token')

In [19]:
myanmardict = {'q':'Rohingya&', 'from':'2017-08-24&', 'language':'en', 'pageSize':100, 'apiKey': token}

In [20]:
myanmar = requests.get('http://newsapi.org/v2/everything?', params=myanmardict)
myanmar

<Response [200]>

In [21]:
myanmarjson = json.loads(myanmar.text)

In [22]:
myanmardf = pd.json_normalize(myanmarjson, record_path = ['articles'])
myanmardf = myanmardf.loc[~myanmardf.title.duplicated(),]

In [23]:
myanmardf.head()

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,None,Global virtual aid conference to be held for R...,Global virtual aid conference to be held for R...,https://uk.reuters.com/article/uk-myanmar-rohi...,None,2020-10-15T07:39:00Z,FILE PHOTO: Rohingyas living in Malaysia prote...,reuters,Reuters
1,Reuters Staff,"Myanmar election app goes offline, has been cr...",A smartphone app produced for Myanmar’s Nov. 8...,https://in.reuters.com/article/myanmar-electio...,https://static.reuters.com/resources/r/?m=02&d...,2020-10-02T15:46:00Z,By Reuters Staff\r\nYANGON (Reuters) - A smart...,reuters,Reuters
2,Reuters Staff,Global virtual aid conference to be held for R...,A global virtual international donor conferenc...,https://www.reuters.com/article/myanmar-rohing...,https://static.reuters.com/resources/r/?m=02&d...,2020-10-15T07:43:00Z,By Reuters Staff\r\n(Reuters) - A global virtu...,reuters,Reuters
3,Reuters Staff,Global virtual aid conference to be held for R...,A global virtual international donor conferenc...,https://in.reuters.com/article/myanmar-rohingy...,https://static.reuters.com/resources/r/?m=02&d...,2020-10-15T08:05:00Z,By Reuters Staff\r\n(Reuters) - A global virtu...,reuters,Reuters
4,"Rozanna Latiff, A. Ananthalakshmi",Anti-migrant sentiment fanned on Facebook in M...,As coronavirus infections surged in Malaysia t...,https://www.reuters.com/article/us-facebook-ma...,https://static.reuters.com/resources/r/?m=02&d...,2020-10-14T03:20:00Z,KUALA LUMPUR (Reuters) - As coronavirus infect...,reuters,Reuters


In [24]:
myanmar_urls = []
for i in range(0, myanmardf.shape[0]):
    myanmar_urls.append(myanmardf.iloc[i]['url'])

In [25]:
len(myanmar_urls)

94

In [26]:
def extractArticles(lst, dic):
    for j in lst:
        try:
            art = Article(j)
            art.download()
            art.parse()
            text = re.sub('\s+',' ',art.text)
            if text not in dic.values():
                dic[j] = text
        except:
            continue

In [27]:
myanmar_articles = {}
extractArticles(myanmar_urls, myanmar_articles)

In [28]:
myanmar_texts = pd.DataFrame.from_dict(myanmar_articles, orient='index', columns=['text'])

In [29]:
myanmar_texts['text']= myanmar_texts['text'].map(lambda x: re.sub('[,\.!?]', '', x))
myanmar_texts['text'] = myanmar_texts['text'].map(lambda x: x.lower())

In [31]:
%store myanmar_texts

Stored 'myanmar_texts' (DataFrame)


In [32]:
%store -r nyt_urls_df

In [33]:
nyt_urls_df.rename({0:'url'}, axis=1, inplace=True)

In [34]:
nyt_urls = []
for i in range(0, nyt_urls_df.shape[0]):
    nyt_urls.append(nyt_urls_df.iloc[i]['url'])

In [35]:
len(nyt_urls)

1059

In [36]:
nyt_articles = {}
extractArticles(nyt_urls, nyt_articles)

In [37]:
nyt_texts = pd.DataFrame.from_dict(nyt_articles, orient='index', columns=['text'])

In [38]:
nyt_texts['text']= nyt_texts['text'].map(lambda x: re.sub('[,\.!?]', '', x))
nyt_texts['text'] = nyt_texts['text'].map(lambda x: x.lower())

In [39]:
nyt_texts.head()

,text
https://www.nytimes.com/2020/10/17/world/asia/rohingya-child-brides.html,bangkok — haresa counted the days by the moon ...
https://www.nytimes.com/2020/10/12/world/asia/myanmar-thailand-gods-army-htoo-twins.html,as they escape to farther and colder climates ...
https://www.nytimes.com/2020/09/22/world/asia/un-general-assembly-live-updates.html,us-china brawl sharpens on the global stage th...
https://www.nytimes.com/2020/09/18/opinion/facebook-democracy.html,for years myanmar’s military used facebook to ...
https://www.nytimes.com/2020/09/15/world/United-Nations-General-Assembly.html,worldwide contagion the worst economic crisis ...


In [40]:
%store nyt_texts

Stored 'nyt_texts' (DataFrame)
